In [ ]:
from azureml.core import Workspace

In [ ]:
ws= Workspace.create(name='Azureml-SDK-brillio',
                     subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad",
                     resource_group="RG-MSFT-ML-POC-01",
                     create_resource_group=True,
                     location="")
ws.write_config(path="./config")

In [ ]:
ws=Workspace.from_config("./config")

In [ ]:
from azureml.core import Datastore
az_store = Datastore.register_azure_blob_container(workspace=ws,
                                                    datastore_name="azure_sdk_blob01",
                                                    account_name="vikaspoctitanic",
                                                    container_name="msbrilliopoc",
                                                    account_key="be/0iIZOayOkNh4ouG4Z6S4JR7fgVM9+R2bpLRcMwO3Z5mne0GEas/ZUNLDldRYqo4/oPDcatLge+AStdxl7hQ==")

In [ ]:
az_store= Datastore.get(ws,"azure_sdk_blob01")

In [ ]:
from azureml.core import Dataset
csv_path= [(az_store,"titanic.csv")]
dataset= Dataset.Tabular.from_delimited_files(path=csv_path)
#register the dataset
dataset = dataset.register(workspace=ws,
                            name="Titanic_vikas",
                            create_new_version=True)

In [ ]:
ws_list=Workspace.list(subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad")
ws_list=list(ws_list)
az_default_store= ws.get_default_datastore()
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
ds_list=list(ws.datasets.keys())
for items in ds_list:
    print(items)

In [ ]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [ ]:
az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(dataframe=df_sub,
                                                          target=az_store,
                                                          name="Titanic_selected_columns")

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-explain-model','interpret-community'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster= ws.compute_targets[cluster_name]
print(cluster," compute cluster found.. using it....")

In [ ]:
from azureml.core.compute import ComputeTarget
compute_cluster= ComputeTarget.create(ws, cluster_name, compute_config)
compute_cluster.wait_for_completion()

In [ ]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration()
run_config.target= cluster_name
run_config.environment = myenv 

In [ ]:
from azureml.pipeline.steps import PythonScriptStep 
from azureml.pipeline.core import PipelineData 
input_ds= ws.datasets.get("Titanic_selected_columns")
datafolder= PipelineData('datafolder', datastore=ws.get_default_datastore())

In [ ]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_exp1_vikas")

In [ ]:
dataprep_step3= PythonScriptStep(name="Datapreparation",
                                source_directory="Training_python_scripts/",
                                script_name= "Dataprep_pipeline.py",
                                inputs=[input_ds.as_named_input('raw_data')],
                                outputs=[datafolder],
                                runconfig=run_config,
                                arguments=['--datafolder', datafolder])

In [ ]:
train_step3= PythonScriptStep(name="ModelTraining",
                             source_directory="Training_python_scripts/",
                             script_name="Training_pipeline_explain.py",
                             inputs=[datafolder],
                             runconfig=run_config,
                             arguments=['--datafolder',datafolder])

In [ ]:
steps= [dataprep_step3,train_step3]

In [ ]:
from azureml.pipeline.core import Pipeline 
new_pipeline = Pipeline(workspace=ws, steps=steps)

In [ ]:
new_experiment= Experiment(workspace=ws, name="pipelineexp01")
pipeline_exp_run= new_experiment.submit(new_pipeline)
pipeline_exp_run.wait_for_completion(show_output=True)